In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
cd /content/drive/MyDrive/Human-Path-Prediction-master (1)

/content/drive/MyDrive/Human-Path-Prediction-master (1)


In [2]:
cd ynet

/content/drive/MyDrive/Human-Path-Prediction-master (1)/ynet


In [3]:
!ls

 config				   requirements.txt
 data				   segmentation_models
'evaluate_inD_longterm(1).ipynb'   test.py
 evaluate_inD_longterm.ipynb	   train_inD_longterm.ipynb
 evaluate_SDD_longterm.ipynb	   train.py
 evaluate_SDD_trajnet.ipynb	   train_SDD_longterm.ipynb
 images				   train_SDD_trajnet.ipynb
 model.py			   utils
 pretrained_models		   w_andb
 __pycache__			   weights_and_biases.py
 README.md


In [6]:
!pip install wandb

     |████████████████████████████████| 1.7 MB 4.2 MB/s 
     |████████████████████████████████| 142 kB 48.2 MB/s 
     |████████████████████████████████| 97 kB 7.2 MB/s 
     |████████████████████████████████| 180 kB 46.4 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=76ea6c8faa5ba639ebbde70fc2ce2da70431f112c6fe47bf83c9e1f23a1ce7b9
  Stored in directory: /root/.cache/pip/wheels/50/ca/fa/8fca8d246e64f19488d07567547ddec8eb084e8c0d7a59226a
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=523fb5571d627ed0e800130edc77ecea7079cfb2bc3445a327bb40954fea2b18
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built subprocess32 pathtools


In [4]:
import pandas as pd
import yaml
import argparse
import torch
from model import YNet, YNetEncoder
import weights_and_biases as wandb

In [5]:
%load_ext autoreload
%autoreload 2

#### Some hyperparameters and settings

In [25]:
CONFIG_FILE_PATH = 'config/sdd_trajnet.yaml'  # yaml config file containing all the hyperparameters
EXPERIMENT_NAME = 'eth'  # arbitrary name for this experiment
DATASET_NAME = 'eth'

TRAIN_DATA_PATH = 'data/eth_ucy/df_short_train.pkl'
TRAIN_IMAGE_PATH = 'data/eth_ucy/train'
VAL_DATA_PATH = 'data/eth_ucy/df_short_test.pkl'
VAL_IMAGE_PATH = 'data/eth_ucy/test'
OBS_LEN = 8  # in timesteps
PRED_LEN = 20  # in timesteps
NUM_GOALS = 20  # K_e
NUM_TRAJ = 1  # K_a

BATCH_SIZE = 4

#### Load config file and print hyperparameters

In [10]:
pip install -r requirements.txt

     |████████████████████████████████| 67 kB 2.7 MB/s 
     |████████████████████████████████| 269 kB 8.6 MB/s 
     |████████████████████████████████| 753.2 MB 14 kB/s 
     |████████████████████████████████| 10.1 MB 22.2 MB/s 
     |████████████████████████████████| 49.4 MB 88.3 MB/s 
     |████████████████████████████████| 25.9 MB 1.2 MB/s 
     |████████████████████████████████| 42 kB 1.1 MB/s 
     |████████████████████████████████| 58 kB 5.9 MB/s 
     |████████████████████████████████| 23.3 MB 48.1 MB/s 
     |████████████████████████████████| 23.3 MB 1.2 MB/s 
     |████████████████████████████████| 22.1 MB 386 kB/s 
     |████████████████████████████████| 22.1 MB 64 kB/s 
     |████████████████████████████████| 17.4 MB 46.7 MB/s 
     |████████████████████████████████| 17.3 MB 24.4 MB/s 
     |████████████████████████████████| 12.8 MB 39.5 MB/s 
     |████████████████████████████████| 12.7 MB 38.4 MB/s 
     |████████████████████████████████| 11.8 MB 28.3 MB/s 
     |████████

In [26]:
with open(CONFIG_FILE_PATH) as file:
    params = yaml.load(file, Loader=yaml.FullLoader)
experiment_name = CONFIG_FILE_PATH.split('.yaml')[0].split('config/')[1]
params

{'CWS_params': 'None',
 'batch_size': 8,
 'decoder_channels': [64, 64, 64, 32, 32],
 'encoder_channels': [32, 32, 64, 64, 64],
 'kernlen': 31,
 'learning_rate': 0.0001,
 'loss_scale': 1000,
 'nsig': 4,
 'num_epochs': 300,
 'rel_threshold': 0.01,
 'resize': 0.25,
 'segmentation_model_fp': 'segmentation_models/SDD_segmentation.pth',
 'semantic_classes': 6,
 'temperature': 1.0,
 'unfreeze': 150,
 'use_CWS': False,
 'use_TTST': True,
 'use_features_only': False,
 'viz_epoch': 10,
 'waypoints': [11]}

#### Load preprocessed Data

In [27]:
!pip3 install pickle5
#df_train = pd.read_pickle(TRAIN_DATA_PATH)
#df_val = pd.read_pickle(VAL_DATA_PATH)

import pickle5 as pickle 
with open(TRAIN_DATA_PATH, "rb") as fh:
    df_train = pickle.load(fh)
with open(VAL_DATA_PATH, "rb") as fh1:
    df_val = pickle.load(fh1)

In [28]:
print(VAL_DATA_PATH)

data/eth_ucy/df_short_test.pkl


In [29]:
print(type(df_val))
print(type(df_train))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


#### Initiate model and load pretrained weights

In [30]:
model = YNet(obs_len=OBS_LEN, pred_len=PRED_LEN, params=params)

/usr/local/lib/python3.7/dist-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'segmentation_models_pytorch.encoders.resnet.ResNetEncoder' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.7/dist-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'segmentation_models_pytorch.base.modules.Conv2dReLU' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.7/dist-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'segmentation_models_pytorch.base.modules.Activation' has changed. you can retrieve the original 

In [31]:
!wandb login --relogin
wandb.init_wandb(params.copy(), model.model)

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


wandb: Currently logged in as: agv (use `wandb login --relogin` to force relogin)


In [ ]:
wandb.init_wandb(params.copy(), model.model)

#### Start training
Note, the Val ADE and FDE are without TTST and CWS to save time. Therefore, the numbers will be worse than the final values.

In [ ]:
model.train(df_train, df_val, params, train_image_path=TRAIN_IMAGE_PATH, val_image_path=VAL_IMAGE_PATH,
            experiment_name=EXPERIMENT_NAME, batch_size=BATCH_SIZE, num_goals=NUM_GOALS, num_traj=NUM_TRAJ, 
            device=None, dataset_name=DATASET_NAME)

Preprocess data


OSError: ignored